<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>

<h1 align=center><font size = 5>FILTRAGEM BASEADA EM CONTEÚDO</font></h1>

Os sistemas de recomendação são um conjunto de algoritmos utilizados para recomendar itens aos usuários com base nas informações obtidas do usuário. Esses sistemas tornaram-se onipresentes e podem ser vistos frequentemente em lojas on-line, bancos de dados de filmes e ferramentas de busca de emprego. Neste notebook, estudaremos os sistemas de recomendação baseados em conteúdo e utilizaremos o Python e a biblioteca Pandas para implementar uma versão simples de um sistema de recomendação desse tipo.

### Índice

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li> <p><a href="#ref1">Aquisição dos dados</a></p> </li>
    <li> <p><a href="#ref2">Pré-processamento</a></p> </li>
    <li> <p><a href="#ref3">Filtragem baseada em conteúdo</a></p> </li>
</ul>
<p></p>
</div>
<br>

<a id="ref1"></a>
# Aquisição dos dados

Para adquirir e extrair os dados, basta executar os seguintes scripts bash:

Conjunto de dados obtido de [GroupLens](http://grouplens.org/datasets/movielens/). Vamos baixar o conjunto de dados. Para baixar os dados, utilizaremos o `!wget`. Utilizaremos o `!wget` para baixar os dados a partir do IBM Object Storage.

__Você sabia?__ Ao utilizar o aprendizado de máquina, é provável que trabalhe com conjuntos de dados de grandes dimensões. Como empresa, onde você pode hospedar seus dados? A IBM está oferecendo uma oportunidade única para empresas, com 10 TB de armazenamento no IBM Cloud Object Storage: Inscreva-se agora gratuitamente: [Inscreva-se agora gratuitamente](http://cocl.us/ML0101EN-IBM-Offer-CC)

In [ ]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

Agora está tudo pronto para começarmos a trabalhar com os dados!

<a id="ref2"></a>
# Pré-processamento

Primeiro, tiramos todas as importações do caminho:

In [ ]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Agora, lemos cada arquivo em seus dataframes:

In [ ]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings.csv')
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

Também removemos o ano da coluna __title (título)__, utilizando a função replace do Pandas, e o armazenamos em uma nova coluna __year (ano)__.

In [ ]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

Feito isso, também dividimos os valores da coluna __Genres (gêneros)__ em uma __lista de gêneros__ para simplificar o uso posterior. Para isso, aplicamos a função split string do Python à coluna correta.

In [ ]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

Uma vez que manter os gêneros em um formato de lista não é o ideal para a técnica de sistema de recomendação baseado em conteúdo, usaremos a técnica One Hot Encoding para converter a lista de gêneros em um vetor no qual cada coluna corresponda a um valor possível da característica. Essa codificação é necessária para utilizarmos dados categóricos como entrada. Nesse caso, armazenamos todos os gêneros diferentes em colunas que contenham 1 ou 0. O 1 indica que um filme tem esse gênero e o 0 indica que não. Também armazenamos esse dataframe em outra variável, já que os gêneros não serão importantes para nosso primeiro sistema de recomendação.

In [ ]:
#Copiamos o dataframe de filmes em um novo, pois não precisaremos utilizar a informação sobre o gênero em nosso primeiro caso.
moviesWithGenres_df = movies_df.copy()

#Para cada linha do dataframe, percorremos a lista de gêneros e colocamos um 1 na coluna correspondente
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Preenchemos os valores NaN com 0, para mostrar que o filme não tem o gênero dessa coluna
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

Em seguida, observamos o dataframe de classificações.

In [ ]:
ratings_df.head()

Cada linha do dataframe de classificações tem um ID de usuário associado a pelo menos um filme, uma classificação e uma marca de hora que indica quando o filme foi classificado. Não precisaremos da coluna de marca de hora, então a descartamos para economizar memória.

In [ ]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

<a id="ref3"></a>
# Sistema de recomendação baseado em conteúdo

Agora vejamos como implementar sistemas de recomendação __baseados em conteúdo__, ou __item-item__. Essa técnica busca descobrir quais são os aspectos favoritos de um item para um usuário e, em seguida, recomenda itens que apresentem esses aspectos. Em nosso caso, tentaremos descobrir os gêneros favoritos do usuário de entrada com base nos filmes e nas classificações dadas.

Começamos criando um usuário de entrada ao quem recomendar filmes:

Aviso: para acrescentar mais filmes, basta aumentar a quantidade de elementos em __userInput__. Fique à vontade para acrescentar mais filmes! Só não se esqueça de escrevê-los com letras maiúsculas e se um filme começar com a palavra “The”, como em “The Matrix”, escreva-o da seguinte maneira: ‘Matrix, The’ .


In [ ]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

#### Adicionar um ID de filme ao usuário de entrada
Com o usuário de entrada completo, extraímos os IDs de seus filmes do dataframe de filmes e adicionamos esses IDs ao dataframe de entrada.

Uma maneira de fazê-lo é primeiro filtrar as linhas que contêm o título dos filmes do usuário de entrada e, em seguida, combinar esse subconjunto com o dataframe de entrada. Também descartamos as colunas desnecessárias do dataframe de entrada para economizar memória.

In [ ]:
#Filtramos os filmes pelo título
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Em seguida, combinamos para obter os IDs dos filmes. Está implicitamente combinando por título.
inputMovies = pd.merge(inputId, inputMovies)
#Descartamos do dataframe de entrada as informações que não utilizaremos
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
#Dataframe de entrada final
#Se um filme adicionado antes não aparecer aqui, é possível que não esteja no 
#dataframe original ou tenha sido escrito de modo diferente. Verifique as letras maiúsculas.
inputMovies

Vamos começar aprendendo as preferências do usuário de entrada. Para isso, obteremos o subconjunto de filmes aos quais o usuário de entrada assistiu a partir do dataframe que contém os gêneros definidos com valores binários.

In [ ]:
#Filtramos os filmes a partir do usuário de entrada
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

Só precisaremos da tabela de gêneros, então faremos uma limpeza, redefinindo o índice e removendo as colunas “movieId” (ID do filme), “title” (título), “genres” (gêneros) e “year” (ano).

In [ ]:
#Redefinimos o índice para evitar problemas futuros
userMovies = userMovies.reset_index(drop=True)
#Removemos informações desnecessárias, a fim de liberar memória e evitar problemas
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

Agora estamos prontos para aprender as preferências do usuário de entrada!

Para fazer isso, transformaremos cada gênero em pesos. Podemos utilizar as classificações do usuário de entrada, multiplicá-las na tabela de gêneros desse usuário e, em seguida, somar a tabela resultante por coluna. Essa operação é, na verdade, um produto escalar entre uma matriz e um vetor, então podemos realizá-la simplesmente chamando a função “dot” do Panda.

In [ ]:
inputMovies['rating']

In [ ]:
#Produto escalar para obter os pesos
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#O perfil de usuário
userProfile

Agora, temos os pesos para cada uma das preferências do usuário. Isso é denominado perfil de usuário. Com esse perfil, podemos recomendar filmes que satisfaçam as preferências do usuário.

Para começar, extraímos a tabela de gênero do dataframe original:

In [ ]:
#Agora obtemos os gêneros de todos os filmes de nosso dataframe original
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#E descartamos as informações desnecessárias
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

In [ ]:
genreTable.shape

Com o perfil do usuário de entrada e a lista completa de filmes e seus gêneros em mãos, calcularemos a média ponderada de cada filme a partir do perfil do usuário de entrada e recomendaremos os vinte filmes que mais o satisfazem.

In [ ]:
#Multiplicamos os gêneros pelos pesos e, em seguida, calculamos a média ponderada
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

In [ ]:
#Ordenamos nossas recomendações em ordem decrescente
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Vamos dar uma olhadinha nos valores
recommendationTable_df.head()

Aqui está a tabela de recomendações!

In [ ]:
#Tabela de recomendações final
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

#### Vantagens e desvantagens da filtragem baseada em conteúdo

##### Vantagens
* Aprende as preferências do usuário
* Altamente personalizada para o usuário

##### Desvantagens
* Não leva em conta a opinião de outras pessoas sobre o item, podendo gerar recomendações de baixa qualidade
* A extração dos dados nem sempre é intuitiva
* Nem sempre é óbvio determinar as características do item que o usuário gosta ou não gosta

## Deseja saber mais?

A IBM SPSS Modeler é uma plataforma analítica abrangente que possui muitos algoritmos de aprendizado de máquina. Foi projetada para levar inteligência preditiva às decisões tomadas pelas pessoas, pelos grupos, pelos sistemas e por sua empresa como um todo. Este curso lhe permite acessar uma avaliação gratuita, disponível aqui: [SPSS Modeler](http://cocl.us/ML0101EN-SPSSModeler).

Também é possível utilizar o Watson Studio para executar esses notebooks mais rapidamente com conjuntos de dados maiores. O Watson Studio é a solução de nuvem de vanguarda da IBM para cientistas de dados, construída por cientistas de dados. Com os notebooks Jupyter, RStudio, Apache Spark e outras bibliotecas populares pré-empacotadas na nuvem, o Watson Studio permite que os cientistas de dados colaborem em seus projetos sem a necessidade de instalar nada. Junte-se hoje mesmo à comunidade de usuários do Watson Studio, que cresce cada dia mais, com uma conta gratuita em [Watson Studio](https://cocl.us/ML0101EN_DSX)

### Obrigado por concluir esta lição!

Notebook criado por: <a href = "https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>

<hr>

Copyright &copy; 2018 [Cognitive Class](https://cocl.us/DX0108EN_CC). Este notebook e seu código-fonte são divulgados de acordo com os termos da [Licença do MIT](https://bigdatauniversity.com/mit-license/).​